# pyvoro2 examples

This notebook demonstrates the main inputs/outputs of **pyvoro2**:

- 0D bounded box tessellation (`Box`)
- 3D periodic tessellation (`PeriodicCell`)
- Classic Voronoi (`mode='standard'`)
- Radical / Laguerre (power diagram) (`mode='power'`)
- Custom plane positioning (`mode='plane'`) with three encodings:
  - `PlaneFromRadii`
  - `PlaneMatrix`
  - `PlanePairs`

The examples focus on **inputs and outputs** (data structures), not on visualization.

In [1]:
import numpy as np

from pyvoro2 import (
    Box,
    PeriodicCell,
    PlaneFromRadii,
    PlaneMatrix,
    PlanePairs,
    compute,
)


def summarize_cell(cell, *, max_vertices=5, max_faces=3):
    """Small, stable summary of a cell dict for display."""
    out = {}
    for k in sorted(cell.keys()):
        v = cell[k]
        if k == "vertices" and isinstance(v, list):
            out[k] = {"n": len(v), "head": v[:max_vertices]}
        elif k == "faces" and isinstance(v, list):
            head = v[:max_faces]
            out[k] = {"n": len(v), "head": head}
        elif k == "adjacency" and isinstance(v, list):
            out[k] = {"n": len(v), "head": v[:max_vertices]}
        else:
            out[k] = v
    return out


## 1) 0D tessellation inside a bounding box

`Box` defines an orthogonal, **non-periodic** bounding box. This is the simplest "0D" setting.

In [2]:
pts = np.array(
    [
        [0.0, 0.0, 0.0],
        [2.0, 0.0, 0.0],
        [0.0, 2.0, 0.0],
        [0.0, 0.0, 2.0],
    ],
    dtype=float,
)

box = Box(bounds=((-5.0, 5.0), (-5.0, 5.0), (-5.0, 5.0)))

cells = compute(
    pts,
    domain=box,
    mode="standard",
    return_vertices=True,
    return_faces=True,
    return_adjacency=False,  # keep output small for display
)

len(cells), summarize_cell(cells[0])


(4,
 {'faces': {'n': 6,
   'head': [{'adjacent_cell': 1, 'vertices': [1, 5, 7, 3]},
    {'adjacent_cell': -3, 'vertices': [1, 0, 4, 5]},
    {'adjacent_cell': -5, 'vertices': [1, 3, 2, 0]}]},
  'id': 0,
  'vertices': {'n': 8,
   'head': [[-5.0, -5.0, -5.0],
    [1.0, -5.0, -5.0],
    [-5.0, 1.0, -5.0],
    [1.0, 1.0, -5.0],
    [-5.0, -5.0, 1.0]]},
  'volume': 216.0})

## 2) 3D periodic tessellation (triclinic cell)

`PeriodicCell` represents a fully periodic triclinic unit cell via a coordinate transform.

In [3]:
cell = PeriodicCell(
    vectors=(
        (10.0, 0.0, 0.0),
        (2.0, 9.5, 0.0),
        (1.0, 0.5, 9.0),
    )
)

pts_pbc = np.array(
    [
        [1.0, 1.0, 1.0],
        [5.0, 5.0, 5.0],
        [8.0, 2.0, 7.0],
        [3.0, 9.0, 4.0],
    ],
    dtype=float,
)

cells_pbc = compute(
    pts_pbc,
    domain=cell,
    mode="standard",
    return_vertices=False,
    return_faces=False,
    return_adjacency=False,
)

# In periodic mode, all Voronoi volumes should sum to the unit cell volume.
cell_volume = abs(np.linalg.det(np.array(cell.vectors, dtype=float)))
sum_vol = float(sum(c["volume"] for c in cells_pbc))
cell_volume, sum_vol


(855.0000000000013, 855.0)

## 3) Standard vs radical Voronoi (power diagram)

In `mode='power'`, each point has a radius/weight, and the tessellation becomes a **radical Voronoi** (Laguerre / power diagram).

In [4]:
radii = np.array([0.0, 0.0, 2.0, 0.0], dtype=float)

cells_std = compute(
    pts_pbc,
    domain=cell,
    mode="standard",
    return_vertices=False,
    return_faces=False,
    return_adjacency=False,
)

cells_pow = compute(
    pts_pbc,
    domain=cell,
    mode="power",
    radii=radii,
    return_vertices=False,
    return_faces=False,
    return_adjacency=False,
)

vols_std = [c["volume"] for c in cells_std]
vols_pow = [c["volume"] for c in cells_pow]

vols_std, vols_pow


([204.52350840152917,
  243.35630134069405,
  231.409081979397,
  175.71110827837984],
 [177.66314170369014,
  213.6503389726455,
  307.3025551674562,
  156.38396415620826])

## 4) Plane positioning (`mode='plane'`)

`mode='plane'` lets you shift the separating plane between points away from the midpoint by specifying a pairwise fraction `t_ij`.

To make the effect obvious, use **two points in a box**: the cell volumes will change as the bisector shifts.

In [5]:
pts2 = np.array([[0.0, 0.0, 0.0], [2.0, 0.0, 0.0]], dtype=float)
box2 = Box(bounds=((-10.0, 10.0), (-5.0, 5.0), (-5.0, 5.0)))

# Baseline: standard midplane
cells_mid = compute(
    pts2,
    domain=box2,
    mode="standard",
    return_vertices=False,
    return_faces=False,
    return_adjacency=False,
)
vol_mid = [c["volume"] for c in cells_mid]

# Custom: move plane toward point 0 by setting t_01 < 0.5 (example value)
plane_pairs = PlanePairs(pairs=[(0, 1, 0.25)], default=0.5)
cells_shift = compute(
    pts2,
    domain=box2,
    mode="plane",
    plane=plane_pairs,
    return_vertices=False,
    return_faces=False,
    return_adjacency=False,
)
vol_shift = [c["volume"] for c in cells_shift]

vol_mid, vol_shift


([1100.0, 900.0], [1050.0, 950.0])

## 5) Three ways to encode the same plane policy

Below we encode the **same** pairwise fraction in three different ways:
- `PlanePairs` (sparse)
- `PlaneMatrix` (dense)
- `PlaneFromRadii` (derived from per-point radii)

For two points, choosing radii `R0 : R1 = t : (1 - t)` yields `t_01 = R0 / (R0 + R1)`.

In [6]:
t = 0.25

# (A) PlanePairs
pp = PlanePairs(pairs=[(0, 1, t)], default=0.5)

# (B) PlaneMatrix
M = np.full((2, 2), 0.5, dtype=float)
M[0, 1] = t
M[1, 0] = 1.0 - t
pm = PlaneMatrix(M)

# (C) PlaneFromRadii
# Choose radii so that R0/(R0+R1) == t
r0 = t
r1 = 1.0 - t
pfr = PlaneFromRadii(np.array([r0, r1], dtype=float))

cells_pp = compute(pts2, domain=box2, mode="plane", plane=pp, return_vertices=False, return_faces=False, return_adjacency=False)
cells_pm = compute(pts2, domain=box2, mode="plane", plane=pm, return_vertices=False, return_faces=False, return_adjacency=False)
cells_pfr = compute(pts2, domain=box2, mode="plane", plane=pfr, return_vertices=False, return_faces=False, return_adjacency=False)

([c["volume"] for c in cells_pp],
 [c["volume"] for c in cells_pm],
 [c["volume"] for c in cells_pfr])


([1050.0, 950.0], [1050.0, 950.0], [1050.0, 950.0])

## 6) Inspecting richer outputs (vertices, faces, adjacency)

Set the `return_*` flags to control output size.

In [7]:
from pprint import pprint

cells_full = compute(
    pts,
    domain=box,
    mode="standard",
    return_vertices=True,
    return_faces=True,
    return_adjacency=True,
)

pprint(summarize_cell(cells_full[0]))


{'adjacency': {'head': [[1, 4, 2], [5, 0, 3], [3, 0, 6], [7, 1, 2], [6, 0, 5]],
               'n': 8},
 'faces': {'head': [{'adjacent_cell': 1, 'vertices': [1, 5, 7, 3]},
                    {'adjacent_cell': -3, 'vertices': [1, 0, 4, 5]},
                    {'adjacent_cell': -5, 'vertices': [1, 3, 2, 0]}],
           'n': 6},
 'id': 0,
 'vertices': {'head': [[-5.0, -5.0, -5.0],
                       [1.0, -5.0, -5.0],
                       [-5.0, 1.0, -5.0],
                       [1.0, 1.0, -5.0],
                       [-5.0, -5.0, 1.0]],
              'n': 8},
 'volume': 216.0}
